## __TRABAJO DE INVESTIGACIÓN__ 
***
### __Métodos de Optimización 2022-2__
#### __Profesor__: Cristián Sepulveda S.
#### __Ayudante__: Tomás Lopez A.
#### __Integrantes__:  Estefanía Álvarez M. - Bastián Loyola J. - Carlos Retamales A.

***

#Se definen los datos del problema


In [115]:
#Caso 1
#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 2 #cantidad de camiones
W = [100,120] # capacidad del camión m
w = [0, 10, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0 0 ;1 1;2 2;3 3;4 4]

5×2 Matrix{Int64}:
 0  0
 1  1
 2  2
 3  3
 4  4

In [134]:
#Caso 2
#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 4 #cantidad de camiones
W = [100,120,50,200] # capacidad del camión m
w = [0, 100, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0 0 5 0;1 5 1 1;2 3 2 2;3 4 3 3; 4 4 5 4]

5×4 Matrix{Int64}:
 0  0  5  0
 1  5  1  1
 2  3  2  2
 3  4  3  3
 4  4  5  4

Con camiones

In [135]:
using JuMP, GLPK

# Problema del viajero, en este caso se busca el camino optimo para recoger desde un centro de reciclaje
# reciclaje de n puntos limpios, con m camiones, cada camión tiene una capacidad W
# y cada reciclaje tiene un peso w[i], el costo de transporte de cada reciclaje desde
# cada punto limpio es c[i,j]. 

#Consideraciones

# El camion solo puede pasar por un punto limpio una vez, excepto el punto de origen que puede pasar
# cuantas veces sea necesario
# Si el camion pasa por el punto de origen su capacidad se restablece a W
# Se desea saber la ruta de cada camion y el costo total de transporte

#Se define el modelo
model = Model(GLPK.Optimizer)



#Se definen las variables
#binarias
#camión j que recoge el reciclaje del punto limpio i
@variable(model, x[1:n, 1:m], Bin)


#Función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:m))

#Restricciones

#1.Un camion puede recoger uno o mas reciclajes
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#2.Un reciclaje solo puede ser recogido por uno o mas camiones
@constraint(model, [i in 1:n], sum(x[i,j] for j in 1:m) >= 1)

#3.La capacidad del camion no puede ser sobrepasada
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j])

#4.El camion puede solo pasar una vez por cada punto limpio, excepto el punto de origen
@constraint(model, [i in 2:n, j in 1:m], x[i,j] <= sum(x[k,j] for k in 1:n if k != i))

#5.El camion puede pasar por el punto de origen cuantas veces sea necesario
@constraint(model, [j in 1:m], sum(x[1,j] for i in 1:n) >= 1)

#6.El camion si pasa por el punto de origen su capacidad se restablece a W
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j] + W[j]*x[1,j])

#7. El camion puede pasar de un punto limpio al punto de origen
@constraint(model, [j in 1:m], x[1,j] <= sum(x[i,j] for i in 1:n))

#8. El camion tiene que pasar por el punto de origen al finalizar su recorrido
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#Se resuelve el modelo
optimize!(model)
#Se imprime el resultado
println("Valor óptimo: ", objective_value(model))

#Se imprime la ruta de cada camión
println("Ruta de cada camión")
for j in 1:m
    println("El camión ", j, " sigue la ruta:")
    #Formato
        #Punto 1 -> Punto 2 -> ... -> Punto n
    print("\t")
    for i in 1:n
        if value(x[i,j]) == 1
            print("Punto ", i, " -> ")
        end
    end
    println("Punto 1")
    
end


Valor óptimo: 15.0
Ruta de cada camión
El camión 1 sigue la ruta:
	Punto 1 -> Punto 2 -> Punto 1
El camión 2 sigue la ruta:
	Punto 1 -> Punto 5 -> Punto 1
El camión 3 sigue la ruta:
	Punto 1 -> Punto 3 -> Punto 1
El camión 4 sigue la ruta:
	Punto 1 -> Punto 4 -> Punto 1


Con Camiones y un drone

In [168]:
# Número de camiones, puntos de reciclaje y vuelos del dron
m = 3
n = 5
k = 2

# Capacidad de los camiones y peso de los puntos de reciclaje
W = [50, 80, 150]
w = [20, 30, 10, 15, 25]

# Matrices de costos de transporte
c = [10 20 30 40 50;
     15 25 35 45 55;
     20 15 40 50 60]
d = [2, 3, 4, 5, 6]

# Crear el modelo
model = Model(with_optimizer(GLPK.Optimizer))

# Definir las variables
@variable(model, x[1:m, 1:n], Bin) # variables binarias que indican si el camión i visita el punto j
@variable(model, y[1:n], Bin) # variables binarias que indican si el dron visita el punto j
@variable(model, z[1:m], Bin) # variables binarias que indican si el camión i vuelve al centro de reciclaje
@variable(model, u[1:m], Bin) # variables binarias que indican si el camión i o el dron regresan al centro de reciclaje

# Definir la función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:m, j in 1:n) + sum(d[j]*y[j] for j in 1:n)) # minimizar el costo total de transporte

# Definir las restricciones
# Cada punto de reciclaje debe ser visitado por exactamente un medio de transporte (camión o dron)
@constraint(model, [j in 1:n], sum(x[i,j] for i in 1:m) + y[j] == 1)

# Los camiones deben respetar su capacidad de carga
@constraint(model, [i in 1:m], sum(w[j]*x[i,j] for j in 1:n) - z[i]*W[i] <= W[i])

# El dron solo puede realizar k vuelos
@constraint(model, sum(y[j] for j in 1:n) <= k)

# Los camiones deben regresar al centro de reciclaje para restablecer su capacidad
@constraint(model, [i in 1:m], z[i]*W[i] <= sum(w[j]*x[i,j] for j in 1:n))

# Todos los medios de transporte deben regresar al centro de reciclaje
@constraint(model, sum(u[i] for i in 1:m) + sum(y[j] for j in 1:n) == 1)

# Resolver el modelo
optimize!(model)

# Imprimir el resultado
println("Costo total: ", objective_value(model))

# Recorrer cada camión
for i in 1:m
    println("Camión ", i, ":")
    ruta = [] # lista para guardar la ruta del camión i
    
    # Recorrer cada punto de reciclaje
    for j in 1:n
        if value(x[i,j]) == 1 # si el camión i visita el punto j
            println("- Punto de reciclaje ", j)
            push!(ruta, j) # añadir el punto j a la ruta del camión i
        end
    end
    
    # Si el camión i vuelve al centro de reciclaje, añadirlo a la ruta
    if value(z[i]) == 1
        println("- Centro de reciclaje (origen)")
        push!(ruta, 0) # añadir el centro de reciclaje a la ruta del camión i
    end
    # Imprimir la ruta completa del camión i
    println("Ruta: ", ruta)
end

# Si el dron regresa al centro de reciclaje, imprimirlo
if value(u[1]) == 1
    println("Dron: Centro de reciclaje (origen)")

# Si el dron visita algún punto de reciclaje, imprimirlo
else
    for j in 1:n
        if value(y[j]) == 1
            println("Dron: Punto de reciclaje ", j)
        end
    end
end




Costo total: 100.99999999999999
Camión 1:
- Punto de reciclaje 1
- Punto de reciclaje 3
- Punto de reciclaje 4
Ruta: Any[1, 3, 4]
Camión 2:
Ruta: Any[]
Camión 3:
- Punto de reciclaje 2
Ruta: Any[2]
Dron: Punto de reciclaje 5


Camiones y drones